# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
    * [Crime in Nottingham](#Crime_Nottm)
    * [Venues in Nottingham](#Venue_Nottm)
    * [LSOA Revisited](#LSOA)

## Introduction: Business Problem <a name="introduction"></a>

Nottingham is a medium sized city in the East Midland of the England in the UK.   It has a thriving and lively nightlife with a wide and eclectic range of restaurants, cafes, bars and clubs catering to many different categories of people including the student of two universities.  The City is a chosen weekend destination for groups of young people from all over the UK.

Nottingham also has, like many cities in the UK, crime issues, with parts of the city and its environs exceeding the national crime rate by in excess of 100% (within particular crime categories - see https://www.plumplot.co.uk/Nottingham-violent-crime-statistics.html). 

The requirement in this case is to assess the optimum placement of restaurants / cafes / bars in areas that are more popular for dining and there is regular footfall and to take account of the crime statistics for that location and there the safety of customers and property of the given business.  The data search should happen for the boroughs of Nottingham and the connected urban boroughs of Broxtowe, Gedling and Rushcliffe.  It is intended that this data should be used by people who are looking to set up a hospitality busiess within the environs of Nottingham to help them decide whcih factors that they will take into account when choosing restaurant location.  

## Data <a name="data"></a>
Based on the definition of the problem, we need to breakdown the research into two separate areas - the crime in a particular area and corresponding restaurant activity in that area.

## Crime in Nottingam 2019 <a name="Crime_Nottm"></a>

Based on the definition of the problem, we need to know the following type of information:-
*	The number of crimes in a given location
*	The type of crimes in a given location
*	The frequency of crimes in a given location.

Data about recorded crimes is held for each Police force in the UK in the Police Data Portal (https://data.police.uk/).  The portal does provide an API to access the data but this will only provide records for the specified month – if no month is specified then data for the previous month is returned.  Since it is part of the scope of this analysis to include a view on the frequency of the crimes then we need to collect data on more than one time period - preferably over a year.  

The Police Data portal allows the selection and download of CSV files – selecting the Police Force and the Month that the crime was recorded in - to download.  The data is downloaded in a zip file containing CSV files in separate directories for each selected month.  Since this data cannot be accessed directly from Python, it was decided to download locally data relevant to the greater Nottingham area for the whole of the year 2019 and hold this in DB2 for subsequent querying using python. It was decided that only crime data should be downloaded, and that outcomes and stop-and-search data would not have any significance in the analysis the crime rate.

The format of the CSV is described in the in the table below.  From examination of this data it is clear that various fields will not be required in the analysis and as such the last column of the table shows which of the fields should be retained or discarded for use in this analysis.


| Field           | Meaning                                                  | Discard   |
|:--------------- |:---------------------------------------------------------|:----------|
| Crime Id        | Identifier for the individual crime                      | Discard   |
| Month           | The Year and Month that the crime was reported in        | Retain    |
| Reported By     | The force that provided the data about the crime.        | Retain    |
| Falls within    | Currently the force that provided the data about the crime | Discard      |
| Location | Rough description of the location of the Crime | Discard | 
| Latitiude & Longitude | The anonymised coordinates of the crime. | Retain  |
| LSOA Code & Name | References to the Lower Layer Super Output Area that the anonymised point falls into (see later) | Retain   |
| Crime Type    | Categorisation of the crime                           | Retain     |
| Last Outcome    |  reference to whichever of the outcomes associated with the crime occurred most recently.| Discard   |
| Context   | A field provided for forces to provide additional human-readable data about individual crimes. | Discard   |

Fields that are not to be retained are generally those used to identify the individual crime and as such are of no use in the aggregation of the crime information.
The LSOA information is a uses a standard method (according the UK Office of National Statistics) of grouping the crimes into a recognised geographical location.   In particular the naming format allows human readable information regarding which Borough the crime was committed in.

## LSOA and choice of Geographic Grouping
As has been noted each crime has been recorded with the Lower Layer Super Output Area (LSOA) which is a UK standard format for statistical reporting – further information can be found [here](https://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-areas-december-2011-population-weighted-centroids?geometry=-1.607%2C52.888%2C-0.564%2C53.032) .  As an area for geographic grouping the LSOA is somewhat small and as such a larger area geographical grouping is required. 


## Records to be Discarded
Given that the selection ctiteria are for the Month and Police force, the downloaded CSV files contain data regarding crimes committed outside the Greater Nottingham area.  In this case it was decided that records with LSOA indicating that the crime was committed outside of the urban boroughs in or contiguous to Nottingham (Nottingham, Broxtowe, Gedling and Rushcliffe) should be discarded from DB2 in order to cut down on Storage requirements.

Below is a first look at the raw crime data held in DB2:


In [94]:
import ibm_db
import ibm_db_dbi
import pandas as pd
import numpy as np


DB2 Service Credentials

In [95]:
#Replace the placeholder values with the actuals for Db2 Service Credentials
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net"            # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port = "50000"                    # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "ctz66893"                 # e.g. "abc12345"
dsn_pwd = "bts-x39p7p8fwlm0"  

In [96]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  ctz66893 on host:  dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net


In [97]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

#query statement to retrieve all rows in in the Crime Table, with the MSOA code and object type assigned
selectQuery = "select* from NOTTINGHAM_CRIME_2019"

#retrieve the query results into a pandas dataframe
NOTTM_CRIMEdf = pd.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
NOTTM_CRIMEdf.head()

,MONTH,REPORTED_BY,LONGITUDE,LATITUDE,LOCATION,LSOA_CODE,LSOA_NAME,CRIME_TYPE,LAST_OUTCOME_CATEGORY,CONTEXT
0,2019-01,Nottinghamshire Police,-1.320816,53.032832,On or near Stoney Lane,E01028092,Broxtowe 001A,Criminal damage and arson,Investigation complete; no suspect identified,None
1,2019-01,Nottinghamshire Police,-1.312767,53.043870,On or near Broad Lane,E01028092,Broxtowe 001A,Public order,Unable to prosecute suspect,None
2,2019-01,Nottinghamshire Police,-1.309872,53.040248,On or near Sports/Recreation Area,E01028092,Broxtowe 001A,Violence and sexual offences,Investigation complete; no suspect identified,None
3,2019-01,Nottinghamshire Police,-1.309787,53.032463,On or near Mansfield Road,E01028092,Broxtowe 001A,Violence and sexual offences,Investigation complete; no suspect identified,None
4,2019-01,Nottinghamshire Police,-1.317157,53.044533,On or near Whitehead Drive,E01028093,Broxtowe 001B,Vehicle crime,Investigation complete; no suspect identified,None


Examination of the number of Unique entries for LSOA locations where crimes have occured . . . 

In [5]:
Nottm_LSOA_Unique = NOTTM_CRIMEdf['LSOA_CODE'].unique()
Nottm_LSOA_Unique.shape

(438,)

 . . . shows that there are 438 different areas, which is too much for this analysis.  However, further examination of the UK Office for National Statistics website shows that there is a higher level grouping for geographical areas - MSOAs - Middle Layer Super Output Areas which hold multiple LSOAs - see this [link](https://geoportal.statistics.gov.uk/datasets/middle-layer-super-output-areas-december-2011-population-weighted-centroids?geometry=-1.700%2C52.878%2C-0.657%2C53.023).  The ONS provides a link from the LSOA to the MSOA which can also be loaded in to DB2 and using SQL Joins this can be incorporated into a Nottinham Crime dataframe for 2019. The data loaded here is the link from the LSOA to the MSOA plus further data about the MSOA itself, which may be useful (but see later).   

In [6]:
#query statement to retrieve all rows in in the Crime Table, with the MSOA code and object type assigned
selectQuery = "select NC.MONTH, NC.REPORTED_BY, NC.LONGITUDE, NC.LATITUDE, NC.LOCATION, NC.LSOA_CODE, NC.LSOA_NAME, NC.CRIME_TYPE, NC.LAST_OUTCOME_CATEGORY,\
MS.MSOA01CD, MS.MSOA01NM, MM.objectid \
from NOTTINGHAM_CRIME_2019 as NC join MSOA_LINK as MS on NC.LSOA_CODE = MS.LSOA01CD \
join MSOA_MASTER as MM on MS.MSOA01CD = MM.MSOA11CD"

#retrieve the query results into a pandas dataframe
NOTTM_MSOA_CRIMEdf = pd.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
NOTTM_MSOA_CRIMEdf.head()

,MONTH,REPORTED_BY,LONGITUDE,LATITUDE,LOCATION,LSOA_CODE,LSOA_NAME,CRIME_TYPE,LAST_OUTCOME_CATEGORY,MSOA01CD,MSOA01NM,OBJECTID
0,2019-01,Nottinghamshire Police,-1.303870,53.017443,On or near Supermarket,E01028112,Broxtowe 002C,Public order,Unable to prosecute suspect,E02005851,Broxtowe 002,2405
1,2019-01,Nottinghamshire Police,-1.320816,53.032832,On or near Stoney Lane,E01028092,Broxtowe 001A,Criminal damage and arson,Investigation complete; no suspect identified,E02005850,Broxtowe 001,2404
2,2019-01,Nottinghamshire Police,-1.312767,53.043870,On or near Broad Lane,E01028092,Broxtowe 001A,Public order,Unable to prosecute suspect,E02005850,Broxtowe 001,2404
3,2019-01,Nottinghamshire Police,-1.309872,53.040248,On or near Sports/Recreation Area,E01028092,Broxtowe 001A,Violence and sexual offences,Investigation complete; no suspect identified,E02005850,Broxtowe 001,2404
4,2019-01,Nottinghamshire Police,-1.309787,53.032463,On or near Mansfield Road,E01028092,Broxtowe 001A,Violence and sexual offences,Investigation complete; no suspect identified,E02005850,Broxtowe 001,2404


Looking at the Number of unique MSOA entries for the above data frame . . . 

As can be seen there are thirteen different categories of Crime recorded by the Police for Nottingham in 2019.  Since the scope of the problem is to investigate the potential impact on a resaurant's property or its customers.  Reveiwing the categories, it is understood that all of the categories should be included since all have the potential to harm property or disuade customers from attending the location.

We can also reveiw the number of crimes by category per month

In [7]:
Nottm_MSOA_Unique = NOTTM_MSOA_CRIMEdf['MSOA01NM'].unique()
Nottm_MSOA_Unique.shape

(76,)

 . . . gives a number of 76 which, across the 4 boroughs for Urban Nottingham is more manageable.  This means we can use different Variations of the SQL above to put together different data frames to look at the 2019 crime data in different ways.  The first thing that we should look at is the differnt categoroes of crime that were committed in Nottingham in 2019.

In [8]:
#query statement to retrieve number of crimes Crime Type
selectQuery = "select CRIME_TYPE, count(*) from NOTTINGHAM_CRIME_2019 group by CRIME_TYPE"

#retrieve the query results into a pandas dataframe
NOTTM_CRIME_TYPE_df = pd.read_sql(selectQuery, pconn)


NOTTM_CRIME_TYPE_df.rename (columns = {'2':'Number Crimes'}, inplace = True)
#print just the LNAME for first row in the pandas data frame
NOTTM_CRIME_TYPE_df

,CRIME_TYPE,Number Crimes
0,Anti-social behaviour,19134.0
1,Bicycle theft,1435.0
2,Burglary,4302.0
3,Criminal damage and arson,6594.0
4,Drugs,3640.0
5,Other crime,1718.0
6,Other theft,5926.0
7,Possession of weapons,814.0
8,Public order,4580.0
9,Robbery,841.0


In [9]:
#query statement to retrieve number of crimes  by Crime Type and Month
selectQuery = "select CRIME_TYPE, MONTH, count(*) as COUNT from NOTTINGHAM_CRIME_2019 group by CRIME_TYPE, MONTH"

#retrieve the query results into a pandas dataframe
NOTTM_CRIME_MONTHdf = pd.read_sql(selectQuery, pconn)

NOTTM_CRIME_MONTHdf.rename (columns = {'6':'Number Crimes'}, inplace = True)
NOTTM_CRIME_MONTHdf.head()

#print just the LNAME for first row in the pandas data frame
NOTTM_CRIME_MONTHdf.head()

,CRIME_TYPE,MONTH,COUNT
0,Anti-social behaviour,2019-01,1303.0
1,Bicycle theft,2019-01,103.0
2,Burglary,2019-01,436.0
3,Criminal damage and arson,2019-01,546.0
4,Drugs,2019-01,286.0


Pivot the data to give an improved table of crimes by type per month.

In [10]:
NOTTM_CRIME_MONTH = NOTTM_CRIME_MONTHdf.pivot(index='CRIME_TYPE',columns='MONTH', values='COUNT')
NOTTM_CRIME_MONTH

MONTH,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
CRIME_TYPE,,,,,,,,,,,,
Anti-social behaviour,1303.0,1323.0,1541.0,1636.0,1768.0,1701.0,2027.0,1809.0,1573.0,1768.0,1405.0,1280.0
Bicycle theft,103.0,77.0,118.0,119.0,138.0,138.0,134.0,112.0,159.0,131.0,113.0,93.0
Burglary,436.0,341.0,320.0,344.0,323.0,374.0,339.0,295.0,347.0,473.0,401.0,309.0
Criminal damage and arson,546.0,585.0,534.0,585.0,595.0,508.0,617.0,534.0,522.0,577.0,490.0,501.0
Drugs,286.0,300.0,308.0,250.0,358.0,296.0,304.0,316.0,318.0,326.0,282.0,296.0
Other crime,146.0,129.0,130.0,136.0,157.0,139.0,151.0,160.0,151.0,160.0,146.0,113.0
Other theft,497.0,481.0,542.0,490.0,453.0,522.0,592.0,457.0,482.0,494.0,434.0,482.0
Possession of weapons,76.0,69.0,78.0,71.0,82.0,56.0,58.0,58.0,72.0,80.0,67.0,47.0
Public order,339.0,336.0,374.0,398.0,439.0,421.0,417.0,414.0,354.0,370.0,377.0,341.0


A Total can be added to the end of this dataframe.

In [11]:
NOTTM_CRIME_MONTH['Total'] = NOTTM_CRIME_MONTH.sum(axis=1)
NOTTM_CRIME_MONTH

MONTH,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,Total
CRIME_TYPE,,,,,,,,,,,,,
Anti-social behaviour,1303.0,1323.0,1541.0,1636.0,1768.0,1701.0,2027.0,1809.0,1573.0,1768.0,1405.0,1280.0,19134.0
Bicycle theft,103.0,77.0,118.0,119.0,138.0,138.0,134.0,112.0,159.0,131.0,113.0,93.0,1435.0
Burglary,436.0,341.0,320.0,344.0,323.0,374.0,339.0,295.0,347.0,473.0,401.0,309.0,4302.0
Criminal damage and arson,546.0,585.0,534.0,585.0,595.0,508.0,617.0,534.0,522.0,577.0,490.0,501.0,6594.0
Drugs,286.0,300.0,308.0,250.0,358.0,296.0,304.0,316.0,318.0,326.0,282.0,296.0,3640.0
Other crime,146.0,129.0,130.0,136.0,157.0,139.0,151.0,160.0,151.0,160.0,146.0,113.0,1718.0
Other theft,497.0,481.0,542.0,490.0,453.0,522.0,592.0,457.0,482.0,494.0,434.0,482.0,5926.0
Possession of weapons,76.0,69.0,78.0,71.0,82.0,56.0,58.0,58.0,72.0,80.0,67.0,47.0,814.0
Public order,339.0,336.0,374.0,398.0,439.0,421.0,417.0,414.0,354.0,370.0,377.0,341.0,4580.0


It is also possible to veiw the monthly crimes for the MSOA locations.  Through the SQL call to DB2:

In [12]:
#query statement to number of crimes by MSOA and Month
selectQuery = "select MS.MSOA01NM, MM.objectid, MS.MSOA01CD, NC.MONTH, count(*) as COUNT \
from NOTTINGHAM_CRIME_2019 as NC join MSOA_LINK as MS on NC.LSOA_CODE = MS.LSOA01CD \
join MSOA_MASTER as MM on MS.MSOA01CD = MM.MSOA11CD group by MS.MSOA01NM, MM.objectid, MS.MSOA01CD, NC.MONTH"

#retrieve the query results into a pandas dataframe
NOTTM_CRIME_MSOA_MONTHdf = pd.read_sql(selectQuery, pconn)

NOTTM_CRIME_MSOA_MONTHdf.rename (columns = {'5':'Number Crimes'}, inplace = True)
#print just the LNAME for first row in the pandas data frame
NOTTM_CRIME_MSOA_MONTHdf.head()

,MSOA01NM,OBJECTID,MSOA01CD,MONTH,COUNT
0,Rushcliffe 014,889,E02005919,2019-01,21.0
1,Rushcliffe 014,889,E02005919,2019-02,17.0
2,Rushcliffe 014,889,E02005919,2019-03,24.0
3,Rushcliffe 014,889,E02005919,2019-04,38.0
4,Rushcliffe 014,889,E02005919,2019-05,30.0


Pivot the data to give an improved table of crimes by MSOA per month.

In [13]:
NOTTM_CRIME_MSOA_MONTH = NOTTM_CRIME_MSOA_MONTHdf.pivot(index='MSOA01NM', columns='MONTH', values='COUNT')
NOTTM_CRIME_MSOA_MONTH['Total'] = NOTTM_CRIME_MSOA_MONTH.sum(axis=1)
NOTTM_CRIME_MSOA_MONTH.head(10)

MONTH,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,Total
MSOA01NM,,,,,,,,,,,,,
Broxtowe 001,33.0,32.0,35.0,45.0,61.0,37.0,43.0,51.0,44.0,35.0,36.0,45.0,497.0
Broxtowe 002,83.0,102.0,95.0,81.0,69.0,80.0,87.0,71.0,76.0,57.0,93.0,108.0,1002.0
Broxtowe 003,58.0,43.0,45.0,36.0,61.0,70.0,57.0,50.0,49.0,66.0,45.0,29.0,609.0
Broxtowe 004,53.0,50.0,63.0,70.0,71.0,49.0,48.0,45.0,51.0,53.0,56.0,43.0,652.0
Broxtowe 007,27.0,21.0,40.0,39.0,33.0,29.0,29.0,23.0,25.0,23.0,27.0,12.0,328.0
Broxtowe 008,58.0,31.0,62.0,66.0,54.0,66.0,66.0,78.0,46.0,60.0,48.0,53.0,688.0
Broxtowe 009,37.0,23.0,40.0,38.0,53.0,32.0,51.0,45.0,42.0,32.0,32.0,29.0,454.0
Broxtowe 010,86.0,55.0,75.0,60.0,77.0,92.0,99.0,57.0,63.0,68.0,59.0,70.0,861.0
Broxtowe 011,164.0,120.0,135.0,126.0,95.0,128.0,114.0,88.0,96.0,91.0,120.0,111.0,1388.0


Whilst useful for the detailed purposes of this analysis, the data can be more easily visualised if displayed for the overall Borough.  The Borough can be found by taking the first part of the Name of the MSOA.

In [14]:
#query statement to number of crimes by MSOA and Month using only the first 4 characters of MSOA Name to identify the borough. 
selectQuery = "select LEFT(MS.MSOA01NM,4) as BOROUGH, NC.MONTH, count(*) as COUNT \
from NOTTINGHAM_CRIME_2019 as NC join MSOA_LINK as MS on NC.LSOA_CODE = MS.LSOA01CD \
join MSOA_MASTER as MM on MS.MSOA01CD = MM.MSOA11CD group by LEFT(MS.MSOA01NM,4), NC.MONTH"

#retrieve the query results into a pandas dataframe
NOTTM_CRIME_BORO_MONTHdf = pd.read_sql(selectQuery, pconn)

# Replace the first four letters with the full name
NOTTM_CRIME_BORO_MONTHdf["BOROUGH"].replace("Brox", "Broxtowe", inplace = True)
NOTTM_CRIME_BORO_MONTHdf["BOROUGH"].replace("Gedl", "Gedling", inplace = True)
NOTTM_CRIME_BORO_MONTHdf["BOROUGH"].replace("Nott", "Nottingham", inplace = True)
NOTTM_CRIME_BORO_MONTHdf["BOROUGH"].replace("Rush", "Rushcliffe", inplace = True)

NOTTM_CRIME_BORO_MONTHdf.rename (columns = {'5':'Number Crimes'}, inplace = True)
#print just the LNAME for first row in the pandas data frame
NOTTM_CRIME_BORO_MONTHdf.head()

,BOROUGH,MONTH,COUNT
0,Broxtowe,2019-01,737.0
1,Broxtowe,2019-02,667.0
2,Broxtowe,2019-03,766.0
3,Broxtowe,2019-04,719.0
4,Broxtowe,2019-05,732.0


In [15]:
NOTTM_CRIME_BORO_MONTH = NOTTM_CRIME_BORO_MONTHdf.pivot(index='BOROUGH',columns='MONTH', values='COUNT')
NOTTM_CRIME_BORO_MONTH['Total'] = NOTTM_CRIME_BORO_MONTH.sum(axis=1)
NOTTM_CRIME_BORO_MONTH.head(10)

MONTH,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,Total
BOROUGH,,,,,,,,,,,,,
Broxtowe,737.0,667.0,766.0,719.0,732.0,772.0,797.0,685.0,683.0,655.0,665.0,629.0,8507.0
Gedling,628.0,624.0,701.0,700.0,702.0,711.0,755.0,693.0,616.0,710.0,635.0,607.0,8082.0
Nottingham,2981.0,2907.0,3250.0,3173.0,3305.0,3286.0,3566.0,3216.0,3057.0,3386.0,3030.0,2762.0,37919.0
Rushcliffe,505.0,472.0,518.0,590.0,608.0,602.0,732.0,596.0,606.0,595.0,585.0,496.0,6905.0


Similarly, it is possible to load information regarding the MSOA number of Crimes by Crime type, and also summarise that by Borough. 

In [99]:
#query statement to retrieve number of crimes by MSOA and Crime Type
selectQuery = "select MS.MSOA01NM, MM.objectid, MS.MSOA01CD, NC.CRIME_TYPE, count(*) as COUNT \
from NOTTINGHAM_CRIME_2019 as NC join MSOA_LINK as MS on NC.LSOA_CODE = MS.LSOA01CD \
join MSOA_MASTER as MM on MS.MSOA01CD = MM.MSOA11CD \
group by MM.ObjectId, MS.MSOA01CD, MS.MSOA01NM, NC.CRIME_TYPE"

#retrieve the query results into a pandas dataframe
NOTTM_CRIME_MSOA_CTdf = pd.read_sql(selectQuery, pconn)

NOTTM_CRIME_MSOA_CTdf.rename (columns = {'5':'COUNT'}, inplace = True)
#print just the LNAME for first row in the pandas data frame
NOTTM_CRIME_MSOA_CTdf.head()

NOTTM_CRIME_MSOA_CT = NOTTM_CRIME_MSOA_CTdf.pivot_table(index=['MSOA01NM','MSOA01CD'], columns='CRIME_TYPE', values='COUNT')
NOTTM_CRIME_MSOA_CT.replace(np.nan, 0, inplace=True)
NOTTM_CRIME_MSOA_CT['Total'] = NOTTM_CRIME_MSOA_CT.sum(axis=1)
NOTTM_CRIME_MSOA_CT.head()

# List of Crimes by type accross the various MSOAs

,CRIME_TYPE,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences,Total
MSOA01NM,MSOA01CD,,,,,,,,,,,,,,,
Broxtowe 001,E02005850,80.0,2.0,31.0,55.0,4.0,12.0,39.0,3.0,15.0,5.0,99.0,5.0,32.0,115.0,497.0
Broxtowe 002,E02005851,199.0,4.0,60.0,107.0,13.0,20.0,47.0,6.0,52.0,10.0,131.0,9.0,59.0,285.0,1002.0
Broxtowe 003,E02005852,106.0,3.0,37.0,35.0,8.0,7.0,56.0,4.0,32.0,4.0,163.0,4.0,48.0,102.0,609.0
Broxtowe 004,E02005853,146.0,5.0,62.0,56.0,14.0,16.0,37.0,7.0,33.0,10.0,66.0,3.0,52.0,145.0,652.0
Broxtowe 007,E02005856,58.0,17.0,47.0,22.0,5.0,8.0,21.0,3.0,17.0,2.0,0.0,4.0,30.0,94.0,328.0


In [17]:
#query statement to retrieve number of crimes by MSOA and Crime Type
selectQuery = "select left(LSOA_NAME,4) as BOROUGH, CRIME_TYPE, count(*) as COUNT \
from NOTTINGHAM_CRIME_2019 group by LEFT(LSOA_NAME,4), CRIME_TYPE"

selectQuery = "select LEFT(MS.MSOA01NM,4) as BOROUGH, NC.CRIME_TYPE, count(*) as COUNT \
from NOTTINGHAM_CRIME_2019 as NC \
join MSOA_LINK as MS on NC.LSOA_CODE = MS.LSOA01CD \
group by LEFT(MS.MSOA01NM,4), NC.CRIME_TYPE"
#retrieve the query results into a pandas dataframe
NOTTM_CRIME_BORO_CTdf = pd.read_sql(selectQuery, pconn)

# Replace the first four letters with the full name
NOTTM_CRIME_BORO_CTdf["BOROUGH"].replace("Brox", "Broxtowe", inplace = True)
NOTTM_CRIME_BORO_CTdf["BOROUGH"].replace("Gedl", "Gedling", inplace = True)
NOTTM_CRIME_BORO_CTdf["BOROUGH"].replace("Nott", "Nottingham", inplace = True)
NOTTM_CRIME_BORO_CTdf["BOROUGH"].replace("Rush", "Rushcliffe", inplace = True)

NOTTM_CRIME_BORO_CTdf.rename (columns = {'5':'COUNT'}, inplace = True)
#print just the LNAME for first row in the pandas data frame
NOTTM_CRIME_BORO_CTdf.head()

NOTTM_CRIME_BORO_CT = NOTTM_CRIME_BORO_CTdf.pivot(index='BOROUGH', columns='CRIME_TYPE', values='COUNT')
NOTTM_CRIME_BORO_CT['Total'] = NOTTM_CRIME_BORO_CT.sum(axis=1)
NOTTM_CRIME_BORO_CT.head()

CRIME_TYPE,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences,Total
BOROUGH,,,,,,,,,,,,,,,
Broxtowe,1822.0,220.0,612.0,765.0,121.0,169.0,590.0,56.0,457.0,95.0,1127.0,82.0,628.0,2273.0,9017.0
Gedling,1789.0,55.0,540.0,884.0,179.0,212.0,578.0,66.0,409.0,65.0,530.0,70.0,578.0,2503.0,8458.0
Nottingham,9922.0,636.0,1944.0,3481.0,1427.0,970.0,2774.0,436.0,2343.0,456.0,2215.0,581.0,2172.0,11545.0,40902.0
Rushcliffe,1215.0,158.0,603.0,649.0,146.0,147.0,635.0,37.0,308.0,41.0,389.0,50.0,791.0,1736.0,6905.0


Finally we should hold a table summary by MSOA with the Code and Object ID for further use.

In [18]:
#query statement to retrieve number of crimes by MSOA
selectQuery = "select MS.MSOA01CD, MS.MSOA01NM, MM.objectid, count(*) as COUNT \
from NOTTINGHAM_CRIME_2019 as NC join MSOA_LINK as MS on NC.LSOA_CODE = MS.LSOA01CD \
join MSOA_MASTER as MM on MS.MSOA01CD = MM.MSOA11CD \
group by MS.MSOA01CD, MS.MSOA01NM, MM.ObjectId"

#retrieve the query results into a pandas dataframe
NOTTM_CRIME_MSOA = pd.read_sql(selectQuery, pconn)

NOTTM_CRIME_MSOA.rename (columns = {'4':'Number Crimes'}, inplace = True)
#print just the LNAME for first row in the pandas data frame
NOTTM_CRIME_MSOA.head()

,MSOA01CD,MSOA01NM,OBJECTID,COUNT
0,E02005919,Rushcliffe 014,889,328.0
1,E02005866,Gedling 002,2201,618.0
2,E02005864,Broxtowe 015,2202,548.0
3,E02005865,Gedling 001,2203,713.0
4,E02005862,Broxtowe 013,2204,538.0


In [102]:
ibm_db.close(conn)

True

The ONS Geo data website also provides population weighted centroids for Middle Layer Super Output Areas for England and Wales as at 31 December.  This can be selected for a square output area which is set in the url.  The square for Nottingham is from (approximately) Longitude -1.350 Latitude 52.700 (SW corner) to Long. -1.000 Lat 53.200 (NE corner).  A JSON file is returned.

Population Centroids JSON
https://ons-inspire.esriuk.com/arcgis/rest/services/Census_Boundaries/Middle_Super_Output_Areas_December_2011_Centroids/MapServer/0/query?where=1%3D1&outFields=*&geometry=-1.350%2C52.700%2C-1.000%2C53.200&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outSR=4326&f=json

In [92]:
import requests
from pandas.io.json import json_normalize
# Define the URL
url = 'https://ons-inspire.esriuk.com/arcgis/rest/services/Census_Boundaries/Middle_Super_Output_Areas_December_2011_Centroids/MapServer/0/query?where=1%3D1&outFields=*&geometry=-1.350%2C52.700%2C-1.000%2C53.200&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outSR=4326&f=json'

results = requests.get(url).json()


This is quite a complexly structured JSON file.  For the purposes of this analysis it needs to be stripped back to the 'features layer'. 

In [93]:
## assign relevant part of JSON to MSOA_Centr JSON
MSOA_Centrj = results['features']
MSOA_Centrj

[{'attributes': {'objectid': 806,
   'msoa11cd': 'E02005405',
   'msoa11nm': 'North West Leicestershire 009'},
  'geometry': {'x': -1.333692935682093, 'y': 52.72288846666765}},
 {'attributes': {'objectid': 810,
   'msoa11cd': 'E02005403',
   'msoa11nm': 'North West Leicestershire 007'},
  'geometry': {'x': -1.3544944882205507, 'y': 52.73823322621531}},
 {'attributes': {'objectid': 848,
   'msoa11cd': 'E02005397',
   'msoa11nm': 'North West Leicestershire 001'},
  'geometry': {'x': -1.342789487712562, 'y': 52.84162595716862}},
 {'attributes': {'objectid': 857,
   'msoa11cd': 'E02005398',
   'msoa11nm': 'North West Leicestershire 002'},
  'geometry': {'x': -1.2847744877246043, 'y': 52.83302037944029}},
 {'attributes': {'objectid': 889,
   'msoa11cd': 'E02005919',
   'msoa11nm': 'Rushcliffe 014'},
  'geometry': {'x': -1.2101654955911696, 'y': 52.851183371126936}},
 {'attributes': {'objectid': 2242,
   'msoa11cd': 'E02005920',
   'msoa11nm': 'Rushcliffe 015'},
  'geometry': {'x': -1.179912

The simplified JSON should be tranformed into a dataframe.

In [101]:
# tranform MSOA_Centrj into a dataframe
MSOA_Centroids_df = json_normalize(MSOA_Centrj)
MSOA_Centroids_df.head()

,attributes.msoa11cd,attributes.msoa11nm,attributes.objectid,geometry.x,geometry.y
0,E02005405,North West Leicestershire 009,806,-1.333693,52.722888
1,E02005403,North West Leicestershire 007,810,-1.354494,52.738233
2,E02005397,North West Leicestershire 001,848,-1.342789,52.841626
3,E02005398,North West Leicestershire 002,857,-1.284774,52.833020
4,E02005919,Rushcliffe 014,889,-1.210165,52.851183


In [23]:
MSOA_Centroids_df.shape

(148, 5)

The spacial selection of the MSOA centroids has included centroids for MSOAs which are not included in the scope of the problem - that is, they are outside of the Nottingham urban area.  However this means that we can join the Centriods dataframe with the NOTTM_CRIME_MSOA data frame.  To start with we need to make sure that they have a common index.

In [24]:
MSOA_Centroids_df.rename(columns={'attributes.msoa11cd':'MSOA01CD', 'attributes.msoa11nm':'MSOA Name', 'attributes.objectid':'OBJECTID','geometry.x':'x','geometry.y':'y'}, inplace=True)
#MSOA_Centroids_df.set_index('MSOA01CD', inplace=True)
MSOA_Centroids_df.head()

,MSOA01CD,MSOA Name,OBJECTID,x,y
0,E02005405,North West Leicestershire 009,806,-1.333693,52.722888
1,E02005403,North West Leicestershire 007,810,-1.354494,52.738233
2,E02005397,North West Leicestershire 001,848,-1.342789,52.841626
3,E02005398,North West Leicestershire 002,857,-1.284774,52.833020
4,E02005919,Rushcliffe 014,889,-1.210165,52.851183


Compare this to the total Crimes by MSOA list calculated previously.

In [25]:
#NOTTM_CRIME_MSOA.set_index('MSOA01CD', inplace=True)
NOTTM_CRIME_MSOA.head()

,MSOA01CD,MSOA01NM,OBJECTID,COUNT
0,E02005919,Rushcliffe 014,889,328.0
1,E02005866,Gedling 002,2201,618.0
2,E02005864,Broxtowe 015,2202,548.0
3,E02005865,Gedling 001,2203,713.0
4,E02005862,Broxtowe 013,2204,538.0


Merge both dataframes on the MSOA code - MSOA01CD, which is common to both.

In [26]:
NOTTM_CRIME_MSOA_CENT = pd.merge(NOTTM_CRIME_MSOA, MSOA_Centroids_df, on='MSOA01CD', how='left')

NOTTM_CRIME_MSOA_CENT.head()


,MSOA01CD,MSOA01NM,OBJECTID_x,COUNT,MSOA Name,OBJECTID_y,x,y
0,E02005919,Rushcliffe 014,889,328.0,Rushcliffe 014,889.0,-1.210165,52.851183
1,E02005866,Gedling 002,2201,618.0,Gedling 002,2201.0,-1.081979,53.038427
2,E02005864,Broxtowe 015,2202,548.0,Broxtowe 015,2202.0,-1.257279,52.908062
3,E02005865,Gedling 001,2203,713.0,Gedling 001,2203.0,-1.165054,53.084129
4,E02005862,Broxtowe 013,2204,538.0,Broxtowe 013,2204.0,-1.244654,52.918488


In [27]:
NOTTM_CRIME_MSOA_CENT.dtypes

MSOA01CD       object
MSOA01NM       object
OBJECTID_x      int64
COUNT         float64
MSOA Name      object
OBJECTID_y    float64
x             float64
y             float64
dtype: object

In [28]:
NOTTM_CRIME_MSOA_CENT.shape

(76, 8)

Clean this up - We only need one Object ID and one MSOA Name.

In [29]:
NOTTM_CRIME_MSOA_CENT.rename(columns={'OBJECTID_x':'OBJECTID'}, inplace=True)
NOTTM_CRIME_MSOA_CENT.drop(['OBJECTID_y','MSOA01NM'], axis=1, inplace=True)
NOTTM_CRIME_MSOA_CENT.head()

,MSOA01CD,OBJECTID,COUNT,MSOA Name,x,y
0,E02005919,889,328.0,Rushcliffe 014,-1.210165,52.851183
1,E02005866,2201,618.0,Gedling 002,-1.081979,53.038427
2,E02005864,2202,548.0,Broxtowe 015,-1.257279,52.908062
3,E02005865,2203,713.0,Gedling 001,-1.165054,53.084129
4,E02005862,2204,538.0,Broxtowe 013,-1.244654,52.918488


We can also join the Dataframe with the breakdown by MSOA and Crime type (NOTTM_CRIME_MSOA_CT) with the Centroids data (MSOA_Centroids_df).

In [30]:
NOTTM_CRIME_MSOA_CT1 = NOTTM_CRIME_MSOA_CT
NOTTM_CRIME_MSOA_CT.sort_values(by=['MSOA01NM'], inplace=True)
#NOTTM_CRIME_MSOA_CT1.rename(columns={'MSOA01NM':'MSOA Name'}, inplace=True)
NOTTM_CRIME_MSOA_CT1.head()

,CRIME_TYPE,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences,Total
MSOA01NM,MSOA01CD,,,,,,,,,,,,,,,
Broxtowe 001,E02005850,80.0,2.0,31.0,55.0,4.0,12.0,39.0,3.0,15.0,5.0,99.0,5.0,32.0,115.0,497.0
Broxtowe 002,E02005851,199.0,4.0,60.0,107.0,13.0,20.0,47.0,6.0,52.0,10.0,131.0,9.0,59.0,285.0,1002.0
Broxtowe 003,E02005852,106.0,3.0,37.0,35.0,8.0,7.0,56.0,4.0,32.0,4.0,163.0,4.0,48.0,102.0,609.0
Broxtowe 004,E02005853,146.0,5.0,62.0,56.0,14.0,16.0,37.0,7.0,33.0,10.0,66.0,3.0,52.0,145.0,652.0
Broxtowe 007,E02005856,58.0,17.0,47.0,22.0,5.0,8.0,21.0,3.0,17.0,2.0,0.0,4.0,30.0,94.0,328.0


Copy the MSOA Centroids dataframe and sort by the MSOA code.

In [31]:
NOTTM_CRIME_MSOA_CENT1 = NOTTM_CRIME_MSOA_CENT
NOTTM_CRIME_MSOA_CENT1.sort_values(by=['MSOA01CD'], inplace=True)
NOTTM_CRIME_MSOA_CENT1.head()

,MSOA01CD,OBJECTID,COUNT,MSOA Name,x,y
44,E02002868,2395,631.0,Nottingham 001,-1.175087,53.009121
46,E02002869,2397,2035.0,Nottingham 002,-1.198104,53.007202
69,E02002871,2462,978.0,Nottingham 004,-1.188205,52.998566
67,E02002872,2460,1310.0,Nottingham 005,-1.207894,52.997246
68,E02002873,2461,776.0,Nottingham 006,-1.165054,52.995386


Merge the two dataframes together.  Rename some of the columns for possible furture use.

In [32]:
NOTTM_CRIME_MSOA_CAT = pd.merge(NOTTM_CRIME_MSOA_CENT1, NOTTM_CRIME_MSOA_CT1, on='MSOA01CD', how='left')
NOTTM_CRIME_MSOA_CAT.drop(['COUNT'], axis=1, inplace=True)
NOTTM_CRIME_MSOA_CAT.rename(columns={'MSOA01CD':'MSOA11CD', 'OBJECTID':'objectid'}, inplace=True)
NOTTM_CRIME_MSOA_CAT#.head()

,MSOA11CD,objectid,MSOA Name,x,y,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences,Total
0,E02002868,2395,Nottingham 001,-1.175087,53.009121,127.0,12.0,24.0,75.0,18.0,15.0,41.0,7.0,48.0,4.0,50.0,4.0,31.0,175.0,631.0
1,E02002869,2397,Nottingham 002,-1.198104,53.007202,438.0,19.0,64.0,162.0,52.0,47.0,164.0,23.0,122.0,23.0,335.0,18.0,79.0,489.0,2035.0
2,E02002871,2462,Nottingham 004,-1.188205,52.998566,222.0,7.0,44.0,99.0,26.0,13.0,67.0,5.0,81.0,11.0,34.0,8.0,31.0,330.0,978.0
3,E02002872,2460,Nottingham 005,-1.207894,52.997246,285.0,17.0,49.0,158.0,55.0,36.0,70.0,24.0,86.0,14.0,4.0,5.0,81.0,426.0,1310.0
4,E02002873,2461,Nottingham 006,-1.165054,52.995386,169.0,2.0,23.0,123.0,31.0,25.0,29.0,8.0,42.0,5.0,1.0,4.0,20.0,294.0,776.0
5,E02002874,2458,Nottingham 007,-1.147781,52.989576,158.0,16.0,35.0,100.0,39.0,54.0,75.0,13.0,48.0,5.0,18.0,9.0,53.0,437.0,1060.0
6,E02002875,2459,Nottingham 008,-1.176846,52.988949,315.0,9.0,66.0,91.0,23.0,24.0,80.0,11.0,74.0,6.0,36.0,8.0,49.0,394.0,1186.0
7,E02002876,2456,Nottingham 009,-1.188537,52.983089,284.0,14.0,54.0,99.0,46.0,27.0,112.0,22.0,100.0,15.0,18.0,16.0,80.0,373.0,1260.0
8,E02002877,2457,Nottingham 010,-1.145603,52.980810,168.0,14.0,43.0,70.0,19.0,18.0,89.0,8.0,48.0,7.0,143.0,11.0,79.0,197.0,914.0
9,E02002878,2466,Nottingham 011,-1.212688,52.979902,249.0,14.0,49.0,110.0,47.0,30.0,77.0,11.0,89.0,13.0,48.0,8.0,46.0,463.0,1254.0


In [33]:
NOTTM_CRIME_MSOA_CAT.shape

(76, 20)

Delete row where the x,y coordinates are NaN.  This is where the geographic selection of the MSOA has cut off the Southern MSOAs for the borough of Rushcliffe - whcih extents to the countryside South of the Nottingham urban area.  It is acceptable to loose these centriods as, given that they are in a rural area, it woudld be safe to assume that they data regarding them would not influence decision on the placement of a venue within the Urban area. 

In [34]:
NOTTM_CRIME_MSOA_CAT.dropna(subset=["x","y"], axis=0, inplace=True)
NOTTM_CRIME_MSOA_CAT.reset_index(drop=True, inplace=True)
NOTTM_CRIME_MSOA_CAT

,MSOA11CD,objectid,MSOA Name,x,y,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences,Total
0,E02002868,2395,Nottingham 001,-1.175087,53.009121,127.0,12.0,24.0,75.0,18.0,15.0,41.0,7.0,48.0,4.0,50.0,4.0,31.0,175.0,631.0
1,E02002869,2397,Nottingham 002,-1.198104,53.007202,438.0,19.0,64.0,162.0,52.0,47.0,164.0,23.0,122.0,23.0,335.0,18.0,79.0,489.0,2035.0
2,E02002871,2462,Nottingham 004,-1.188205,52.998566,222.0,7.0,44.0,99.0,26.0,13.0,67.0,5.0,81.0,11.0,34.0,8.0,31.0,330.0,978.0
3,E02002872,2460,Nottingham 005,-1.207894,52.997246,285.0,17.0,49.0,158.0,55.0,36.0,70.0,24.0,86.0,14.0,4.0,5.0,81.0,426.0,1310.0
4,E02002873,2461,Nottingham 006,-1.165054,52.995386,169.0,2.0,23.0,123.0,31.0,25.0,29.0,8.0,42.0,5.0,1.0,4.0,20.0,294.0,776.0
5,E02002874,2458,Nottingham 007,-1.147781,52.989576,158.0,16.0,35.0,100.0,39.0,54.0,75.0,13.0,48.0,5.0,18.0,9.0,53.0,437.0,1060.0
6,E02002875,2459,Nottingham 008,-1.176846,52.988949,315.0,9.0,66.0,91.0,23.0,24.0,80.0,11.0,74.0,6.0,36.0,8.0,49.0,394.0,1186.0
7,E02002876,2456,Nottingham 009,-1.188537,52.983089,284.0,14.0,54.0,99.0,46.0,27.0,112.0,22.0,100.0,15.0,18.0,16.0,80.0,373.0,1260.0
8,E02002877,2457,Nottingham 010,-1.145603,52.980810,168.0,14.0,43.0,70.0,19.0,18.0,89.0,8.0,48.0,7.0,143.0,11.0,79.0,197.0,914.0
9,E02002878,2466,Nottingham 011,-1.212688,52.979902,249.0,14.0,49.0,110.0,47.0,30.0,77.0,11.0,89.0,13.0,48.0,8.0,46.0,463.0,1254.0


Now that the we have a breakdown of crimes by MSOA we can veiw that on a map.  First import Folium

In [35]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [37]:
# define the world map centered around Nottingham a low zoom level
# Nottingham latitude and longitude values
Nottm_latitude = 52.952
Nottm_longitude = -1.157
Nottm_map = folium.Map(location=[Nottm_latitude, Nottm_longitude], zoom_start=12)
# Nottm_map

In [38]:
MSOA_Centroids_map = folium.map.FeatureGroup()

# loop through the MSOAs and add each to the incidents feature group
for lat, lng, in zip(NOTTM_CRIME_MSOA_CAT.y, NOTTM_CRIME_MSOA_CAT.x):
    MSOA_Centroids_map.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(NOTTM_CRIME_MSOA_CAT.y)
longitudes = list(NOTTM_CRIME_MSOA_CAT.x)
labels = list(NOTTM_CRIME_MSOA_CAT.MSOA11CD)


for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(Nottm_map)    
       
# add MSOA to map    
Nottm_map.add_child(MSOA_Centroids_map)

## Venues in Nottingam 2019 <a name="Venue_Nottm"></a>

Since we now have the Centroids, we can use them to select infoamtion from the Foursquare API.  Foursquare allows the interogation data concerning venues adjacent to a given geigraphic area.  It also provides 'premium' data to collect more infomation regarding the venues, such as ratings, tips and further information.  Given that this analysis is for the general area we will satrt with the non premium service.   


For the purposes of selection we should use a range of 400 meters around the centroids to look for nearby Venues.  Start off by looking at the Centre of Nottingham, searching for food venues and bars (see https://developer.foursquare.com/docs/build-with-foursquare/categories/)

In [65]:
CLIENT_ID = '4Y0AAKTDSJHFB2VVIAI3R02XCMRTF0RGWPV1WURZFON1ICGV' # your Foursquare ID
CLIENT_SECRET = '4FGESPJTB5IXJ3YCG5CEAJB0Z0VVDXYBT54KWPM5DUGN52EE' # your Foursquare Secret
VERSION = '20180604'
limit = 100
radius = 400
Nottm_latitude = 52.952
Nottm_longitude = -1.150
Nottm_CatId='4d4b7105d754a06374d81259,4d4b7105d754a06376d81259'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
    Nottm_latitude,
    Nottm_longitude,
    radius,
    limit,
    Nottm_CatId)
#url

In [66]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb9b6b0949393001bf22bc3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'City Centre',
  'headerFullLocation': 'City Centre, Nottingham',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 80,
  'suggestedBounds': {'ne': {'lat': 52.955600003600004,
    'lng': -1.1440358868855682},
   'sw': {'lat': 52.94839999639999, 'lng': -1.1559641131144316}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5506c567498ee93a88d9a794',
       'name': 'Five Guys',
       'location': {'address': '35 Long Row',
        'lat': 52.953776733408674,
        'lng': -1.1502915209188131,
        'labeledLatLngs': [{'label': 'display',
 

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [67]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [68]:
venues = results['response']['groups'][0]['items']
    
Nottm_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Nottm_venues =Nottm_venues.loc[:, filtered_columns]

# filter the category for each row
Nottm_venues['venue.categories'] = Nottm_venues.apply(get_category_type, axis=1)

# clean columns
Nottm_venues.columns = [col.split(".")[-1] for col in Nottm_venues.columns]

Nottm_venues.head()

,name,categories,lat,lng
0,Five Guys,Burger Joint,52.953777,-1.150292
1,World Service,Restaurant,52.950793,-1.152312
2,Ludorati Café Bar,Café,52.951050,-1.152066
3,Delilah,Deli / Bodega,52.953189,-1.146546
4,Kitty Cafe,Pet Café,52.952052,-1.152098


Now Explore the MSOAs in the Nottingham Urban Area. Define a function for determine the nearby venues for MSOAs recorded in NOTTM_CRIME_MSOA_CAT.

In [69]:
def getNottmVenues(names, latitudes, longitudes, radius=400):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit,
            Nottm_CatId)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    Nottm_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    Nottm_venues.columns = ['MSOA Name', 
                  'MSOA_Latitude', 
                  'MSOA_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    return(Nottm_venues)

In [70]:
Nottm_venues = getNottmVenues(names=NOTTM_CRIME_MSOA_CAT['MSOA Name'],
                                   latitudes=NOTTM_CRIME_MSOA_CAT['y'],
                                   longitudes=NOTTM_CRIME_MSOA_CAT['x']
                                  )

Nottingham 001
Nottingham 002
Nottingham 004
Nottingham 005
Nottingham 006
Nottingham 007
Nottingham 008
Nottingham 009
Nottingham 010
Nottingham 011
Nottingham 012
Nottingham 013
Nottingham 014
Nottingham 015
Nottingham 016
Nottingham 017
Nottingham 018
Nottingham 019
Nottingham 020
Nottingham 021
Nottingham 022
Nottingham 023
Nottingham 024
Nottingham 025
Nottingham 026
Nottingham 027
Nottingham 029
Nottingham 030
Nottingham 031
Nottingham 032
Nottingham 034
Nottingham 035
Nottingham 036
Nottingham 037
Broxtowe 001
Broxtowe 002
Broxtowe 003
Broxtowe 004
Broxtowe 007
Broxtowe 008
Broxtowe 009
Broxtowe 010
Broxtowe 011
Broxtowe 012
Broxtowe 013
Broxtowe 014
Broxtowe 015
Gedling 001
Gedling 002
Gedling 004
Gedling 005
Gedling 006
Gedling 007
Gedling 008
Gedling 009
Gedling 010
Gedling 011
Gedling 012
Gedling 013
Gedling 014
Gedling 015
Rushcliffe 003
Rushcliffe 004
Rushcliffe 005
Rushcliffe 006
Rushcliffe 007
Rushcliffe 008
Rushcliffe 009
Rushcliffe 011
Rushcliffe 012
Rushcliffe 013
Rus

In [71]:
Nottm_venues.head()

,MSOA Name,MSOA_Latitude,MSOA_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Nottingham 001,53.009121,-1.175087,Smith’s Coffee Shop & Cafe,53.010254,-1.173920,Café
1,Nottingham 001,53.009121,-1.175087,Winnies coffee shop and cafe lounge,53.010106,-1.173332,Café
2,Nottingham 001,53.009121,-1.175087,Sea King Fish Bar,53.010079,-1.173404,Fish & Chips Shop
3,Nottingham 004,52.998566,-1.188205,Apollo Fish Bar,52.996991,-1.192789,Fast Food Restaurant
4,Nottingham 004,52.998566,-1.188205,Highbury Fish Bar,52.995534,-1.191196,Fish & Chips Shop


In [72]:
Nottm_venues.shape

(159, 7)

Reveiw the data on a Map

In [73]:
# create map of Nottingham using latitude and longitude values of Venues Nearby to MSOA
Ven_Map_Nottingham = folium.Map(location=[Nottm_latitude, Nottm_longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(Nottm_venues['Venue_Latitude'], Nottm_venues['Venue_Longitude'], Nottm_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Ven_Map_Nottingham)  
    
Ven_Map_Nottingham

## LSOA Revisited <a name="LSOA"></a>

Closer examination of the map will show that there a very few entries for the centre of Nottingham.  Looking back to the MSOA Centroids map we can note that the we can also see that there are no MSOA Centroids based in the centre of the city and so this means that the search for nearby venues may not have found results for the city centre.  Indeed, if we examine the inital result for the Foursqaure search for the venter of Nottingham, the venues found in that search were not returned for the MSOA search.  This may partially be explained due to the fact that the MSOA centroids provided by the ONS are weighted by population.  The population information is sourced from Census data and reflects the fact that no mayy people live in the Centre of the City, compared tot he suburbs.  At this point it is appropriate to reveiw the approact to finding the point to reveiw the information in order to get better coverage of Nottingham.  Going back to original crime detials held in NOTTM_CRIMEdf we can  see that each crime has an anonimised latitude and longitude associated with it.  It should therefor be possible to generate a crime weighted centroid for an LSOA (or indeed the linked MSOA) by finding the average of latitude and longitde for all crimes commited in the LSOA (MSOA).  Attempt to work this through for an LSOA - this will provide more detial coverage of the city.       

In [74]:
#  Reveiw the dataframe 
NOTTM_CRIMEdf.head()

,MONTH,REPORTED_BY,LONGITUDE,LATITUDE,LOCATION,LSOA_CODE,LSOA_NAME,CRIME_TYPE,LAST_OUTCOME_CATEGORY,CONTEXT
0,2019-01,Nottinghamshire Police,-1.320816,53.032832,On or near Stoney Lane,E01028092,Broxtowe 001A,Criminal damage and arson,Investigation complete; no suspect identified,None
1,2019-01,Nottinghamshire Police,-1.312767,53.043870,On or near Broad Lane,E01028092,Broxtowe 001A,Public order,Unable to prosecute suspect,None
2,2019-01,Nottinghamshire Police,-1.309872,53.040248,On or near Sports/Recreation Area,E01028092,Broxtowe 001A,Violence and sexual offences,Investigation complete; no suspect identified,None
3,2019-01,Nottinghamshire Police,-1.309787,53.032463,On or near Mansfield Road,E01028092,Broxtowe 001A,Violence and sexual offences,Investigation complete; no suspect identified,None
4,2019-01,Nottinghamshire Police,-1.317157,53.044533,On or near Whitehead Drive,E01028093,Broxtowe 001B,Vehicle crime,Investigation complete; no suspect identified,None


So lets reconnect to DB2 and run some more SQL

In [75]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  ctz66893 on host:  dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net


In [76]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

#query statement to retrieve all rows in in the Crime Table, with the MSOA code and object type assigned
selectQuery = "select LSOA_NAME, count(*) as COUNT, AVG(LONGITUDE) as LONGITUDE,  AVG(LATITUDE) as LATITUDE from NOTTINGHAM_CRIME_2019 group by LSOA_NAME"


#retrieve the query results into a pandas dataframe
NOTTM_LSOA_CC = pd.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
NOTTM_LSOA_CC.head()

,LSOA_NAME,COUNT,LONGITUDE,LATITUDE
0,Broxtowe 001A,67.0,-1.314053,53.037770
1,Broxtowe 001B,42.0,-1.314539,53.042886
2,Broxtowe 001C,149.0,-1.304791,53.021653
3,Broxtowe 001D,70.0,-1.298100,53.021100
4,Broxtowe 001E,169.0,-1.294581,53.015193


We can also pull the same tyep of information for the MSOA.

In [77]:
#query statement to retrieve all rows in in the Crime Table, with the MSOA code and object type assigned
selectQuery = "select MS.MSOA01NM, count(*) as COUNT, avg(NC.LONGITUDE) as LONGITUDE, AVG(NC.LATITUDE) as LATITUDE \
from NOTTINGHAM_CRIME_2019 as NC join MSOA_LINK as MS on NC.LSOA_CODE = MS.LSOA01CD \
group by MS.MSOA01NM"
#retrieve the query results into a pandas dataframe
NOTTM_MSOA_CC = pd.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
NOTTM_MSOA_CC.head()

,MSOA01NM,COUNT,LONGITUDE,LATITUDE
0,Broxtowe 001,497.0,-1.302449,53.023346
1,Broxtowe 002,1002.0,-1.306782,53.015598
2,Broxtowe 003,609.0,-1.285894,53.007876
3,Broxtowe 004,652.0,-1.252607,52.999937
4,Broxtowe 005,252.0,-1.271513,52.992906


In [78]:
# create map of Nottingham using latitude and longitude values
LCr_Map_Nottingham = folium.Map(location=[Nottm_latitude, Nottm_longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(NOTTM_LSOA_CC['LATITUDE'], NOTTM_LSOA_CC['LONGITUDE'], NOTTM_LSOA_CC['LSOA_NAME']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(LCr_Map_Nottingham)
    
for lat, lng, label in zip(NOTTM_MSOA_CC['LATITUDE'], NOTTM_MSOA_CC['LONGITUDE'], NOTTM_MSOA_CC['MSOA01NM']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(LCr_Map_Nottingham) 
    
LCr_Map_Nottingham

In the above map the Blue dots are the MSOAs, the blue dots the LSOAs - both wirghted for crime.  From this it would appear that the use of Crime Weighted LSOAs gives better coverage accross Nottingham and includes the city centre. We can elect to use the LSOAs going forward.  

In [80]:
ibm_db.close(conn)   #  so hopefully that shoud be that

Exception: Connection is not active

Re- run the function to get the nearby velues with the LSOA data - set the range to 200 meters since the cenroids are closer together.

In [81]:
Nottm_venues

,MSOA Name,MSOA_Latitude,MSOA_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Nottingham 001,53.009121,-1.175087,Smith’s Coffee Shop & Cafe,53.010254,-1.173920,Café
1,Nottingham 001,53.009121,-1.175087,Winnies coffee shop and cafe lounge,53.010106,-1.173332,Café
2,Nottingham 001,53.009121,-1.175087,Sea King Fish Bar,53.010079,-1.173404,Fish & Chips Shop
3,Nottingham 004,52.998566,-1.188205,Apollo Fish Bar,52.996991,-1.192789,Fast Food Restaurant
4,Nottingham 004,52.998566,-1.188205,Highbury Fish Bar,52.995534,-1.191196,Fish & Chips Shop
5,Nottingham 006,52.995386,-1.165054,Raj Rani,52.992803,-1.168491,Indian Restaurant
6,Nottingham 009,52.983089,-1.188537,Mr T's,52.984096,-1.186810,Fast Food Restaurant
7,Nottingham 010,52.980810,-1.145603,Le Mistral,52.981657,-1.146122,Mediterranean Restaurant
8,Nottingham 010,52.980810,-1.145603,Ginza - Teppan Yaki,52.982506,-1.145782,Japanese Restaurant
9,Nottingham 010,52.980810,-1.145603,The Crimson Tree,52.983305,-1.144951,Breakfast Spot


In [83]:
def getNottmVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit,
            Nottm_CatId)
        
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    Nottm_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    Nottm_venues.columns = ['LSOA_Name', 
                  'LSOA_Latitude', 
                  'LSOA_Longitude',         
                  'Venue',
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    return(Nottm_venues)


Nottm_venues = getNottmVenues(names=NOTTM_LSOA_CC['LSOA_NAME'],
                                   latitudes=NOTTM_LSOA_CC['LATITUDE'],
                                   longitudes=NOTTM_LSOA_CC['LONGITUDE']
                                  )

Broxtowe 001A
Broxtowe 001B
Broxtowe 001C
Broxtowe 001D
Broxtowe 001E
Broxtowe 002A
Broxtowe 002B
Broxtowe 002C
Broxtowe 002D
Broxtowe 003A
Broxtowe 003B
Broxtowe 003C
Broxtowe 003D
Broxtowe 003E
Broxtowe 004A
Broxtowe 004B
Broxtowe 004C
Broxtowe 004D
Broxtowe 004E
Broxtowe 004F
Broxtowe 007A
Broxtowe 007B
Broxtowe 007C
Broxtowe 007D
Broxtowe 007E
Broxtowe 008A
Broxtowe 008B
Broxtowe 008C
Broxtowe 008D
Broxtowe 009A
Broxtowe 009B
Broxtowe 009C
Broxtowe 009D
Broxtowe 009E
Broxtowe 010A
Broxtowe 010B
Broxtowe 010C
Broxtowe 010D
Broxtowe 010E
Broxtowe 010F
Broxtowe 011A
Broxtowe 011B
Broxtowe 011C
Broxtowe 011D
Broxtowe 011E
Broxtowe 012A
Broxtowe 012B
Broxtowe 012C
Broxtowe 012D
Broxtowe 013A
Broxtowe 013B
Broxtowe 013C
Broxtowe 013D
Broxtowe 013E
Broxtowe 014A
Broxtowe 014B
Broxtowe 014C
Broxtowe 014D
Broxtowe 014E
Broxtowe 015A
Broxtowe 015B
Broxtowe 015C
Broxtowe 015D
Broxtowe 015E
Broxtowe 016A
Broxtowe 016B
Broxtowe 016C
Broxtowe 016D
Broxtowe 016E
Broxtowe 016F
Broxtowe 016G
Erewas

In [85]:
Nottm_venues.head()

,LSOA_Name,LSOA_Latitude,LSOA_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Broxtowe 001D,53.021100,-1.298100,Hilltop Pizza,53.020157,-1.296591,Pizza Place
1,Broxtowe 001E,53.015193,-1.294581,Pizza Hut Delivery,53.014380,-1.294511,Pizza Place
2,Broxtowe 001E,53.015193,-1.294581,The Olive Tree,53.014432,-1.294398,Kebab Restaurant
3,Broxtowe 001E,53.015193,-1.294581,Jade Kitchen,53.014484,-1.294160,Chinese Restaurant
4,Broxtowe 002C,53.017924,-1.304420,Greggs,53.017824,-1.304440,Bakery


In [86]:
Nottm_venues.shape

(426, 7)

Again - reveiw this on a map.  Resaurants in yellow, LSOAs in red.

In [89]:
# create map of Nottingham using latitude and longitude values
LCr_Map_Nottingham = folium.Map(location=[Nottm_latitude, Nottm_longitude], zoom_start=13)

# add markers for LSOA to map
for lat, lng, label in zip(NOTTM_LSOA_CC['LATITUDE'], NOTTM_LSOA_CC['LONGITUDE'], NOTTM_LSOA_CC['LSOA_NAME']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(LCr_Map_Nottingham)
    
for lat, lng, label in zip(Nottm_venues['Venue_Latitude'], Nottm_venues['Venue_Longitude'], Nottm_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(LCr_Map_Nottingham) 
    
LCr_Map_Nottingham

In [91]:
print('There are {} unique categories.'.format(len(Nottm_venues['Venue_Category'].unique())))

Nottm_cats = Nottm_venues['Venue_Category'].unique()
Nottm_cats


There are 58 unique categories.


array(['Pizza Place', 'Kebab Restaurant', 'Chinese Restaurant', 'Bakery',
       'Indian Restaurant', 'Italian Restaurant', 'Fish & Chips Shop',
       'Café', 'Fast Food Restaurant', 'Scandinavian Restaurant',
       'Sandwich Place', 'American Restaurant', 'Portuguese Restaurant',
       'Breakfast Spot', 'Korean Restaurant', 'Thai Restaurant',
       'Cafeteria', 'Theme Restaurant', 'Falafel Restaurant',
       'Deli / Bodega', 'Burger Joint', 'BBQ Joint', 'Restaurant', 'Food',
       'Steakhouse', 'Greek Restaurant', 'Snack Place',
       'Cuban Restaurant', 'Mediterranean Restaurant',
       'Japanese Restaurant', 'Eastern European Restaurant',
       'Lebanese Restaurant', 'Asian Restaurant',
       'Vegetarian / Vegan Restaurant', 'Middle Eastern Restaurant',
       'Fried Chicken Joint', 'Food Court', 'Donut Shop', 'Burrito Place',
       'Dim Sum Restaurant', 'Szechuan Restaurant', 'French Restaurant',
       'Caribbean Restaurant', 'Buffet', 'Sushi Restaurant',
       'Noodle

Now the restaurants in Nottingham city centre and its surrounding Urban environment are available - we have a list of venues and associated LSOAs that we can now carry out some analysis on. 